Drew Hill

November 2019

Kaggle Challenge: https://www.kaggle.com/c/bigquery-geotab-intersection-congestion/overview

**Challenge summary**: using BigQuery (BQ) and Big Query Machine Learning (BQML), predict congestion at intersections in Atlanta, Boston, Chicago, Philadelphia

In [5]:
from google.cloud import bigquery
from google.cloud.bigquery import magics

In [13]:
# State google cloud parameters
PROJECT_ID = 'kaggle-bq-ml-drew'
magics.context.project = PROJECT_ID

client = bigquery.Client(project=PROJECT_ID, location="US")
dataset = client.create_dataset('bq_congestion', exists_ok=True)

/Users/lawsonhill/miniconda3/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [11]:
# Take a look at the training data format
table_train = client.get_table("kaggle-competition-datasets.geotab_intersection_congestion.train")
client.list_rows(table_train, max_results = 5).to_dataframe()

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,...,TimeFromFirstStop_p40,TimeFromFirstStop_p50,TimeFromFirstStop_p60,TimeFromFirstStop_p80,DistanceToFirstStop_p20,DistanceToFirstStop_p40,DistanceToFirstStop_p50,DistanceToFirstStop_p60,DistanceToFirstStop_p80,City
0,2079854,33,42.34636,-71.09095,Fenway,Fenway,N,N,8,False,...,31,40,44,59,0.0,25.5,32.3,51.8,68.9,Boston
1,2079857,33,42.34636,-71.09095,Fenway,Fenway,N,N,13,False,...,0,10,17,29,0.0,0.0,9.9,21.8,40.4,Boston
2,2079872,33,42.34636,-71.09095,Fenway,Fenway,N,N,15,False,...,22,33,38,60,0.0,43.2,47.3,51.1,65.7,Boston
3,2079893,33,42.34636,-71.09095,Fenway,Fenway,N,N,12,False,...,0,14,18,31,0.0,0.0,15.6,22.5,44.4,Boston
4,2079896,33,42.34636,-71.09095,Fenway,Fenway,N,N,16,False,...,0,23,34,65,0.0,0.0,26.4,52.9,68.9,Boston


In [15]:
# Jupyter magic command to allow BQ built-on commands
%load_ext google.cloud.bigquery

# Model 1
   * Predict: congestion
   * Loss Function: RMSE
   * Outputs: ID, TARGET
   

In [21]:
# Create model

%%bigquery
CREATE MODEL IF NOT EXISTS `bqml_example.model1`
OPTIONS(model_type='linear_reg') AS
SELECT
    TotalTimeStopped_p20 as label,
    Weekend,
    Hour,
    EntryHeading,
    ExitHeading,
    City
FROM
  `kaggle-competition-datasets.geotab_intersection_congestion.train`
WHERE
    RowId < 2600000

""


In [23]:
# Evaluate training statistics for the model

%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `bqml_example.model1`)
ORDER BY iteration 

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,55.255745,51.769858,None,12453


In [24]:
# Evaluate model performance

%%bigquery
SELECT
  *
FROM ML.EVALUATE(MODEL `bqml_example.model1`, (
  SELECT
    TotalTimeStopped_p20 as label,
    Weekend,
    Hour,
    EntryHeading,
    ExitHeading,
    City
  FROM
    `kaggle-competition-datasets.geotab_intersection_congestion.train`
  WHERE
    RowId > 2600000))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,2.776505,29.772684,1.187835,1.855531,-0.005618,0.000375


In [25]:
# Predict outcomes using model

%%bigquery df
SELECT
  RowId,
  predicted_label as TotalTimeStopped_p20
FROM
  ML.PREDICT(MODEL `bqml_example.model1`,
    (
    SELECT
        RowId,
        Weekend,
        Hour,
        EntryHeading,
        ExitHeading,
        City
    FROM
      `kaggle-competition-datasets.geotab_intersection_congestion.test`))
    ORDER BY RowId ASC

In [26]:
# Output as a dataframe

df['RowId'] = df['RowId'].apply(str) + '_0'
df.rename(columns={'RowId': 'TargetId', 'TotalTimeStopped_p20': 'Target'}, inplace=True)
df

,TargetId,Target
0,0_0,1.876350
1,1_0,1.659791
2,2_0,1.568656
3,3_0,1.699379
4,4_0,1.608244
...,...,...
1920330,1920330_0,1.464732
1920331,1920331_0,1.504320
1920332,1920332_0,1.332445
1920333,1920333_0,1.623085
